<a href="https://colab.research.google.com/github/hyejwon/1_ne/blob/main/%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##자동차 연비 예측하기 : 회귀
##회귀는 가격이나 확률 같이 연속된 출력 값을 예측하는 것이 목적
##이와는 달리 분류는 여러개의 클래스 중 하나의 클래스를 선택하는 것이 목적
##이 노트북은 auto mpg 데이터셋을 사용하여 1970년대 후반과 1980년대 초반의 자동차 연비를 예측하는 모델을 만듦
##이 기간에 출시된 자동차 정보를 모델에 제공. 실린더 수, 배기량, 마력, 공차 중량 같은 속성 

In [ ]:
pip install -q seaborn

In [ ]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

In [ ]:
#데이터 구하기

dataset_path=keras.utils.get_file("auto-mpg-data","http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")

In [ ]:
#판다스를 사용하여 데이터를 읽음

column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration','Model Year','Origin']
raw_dataset = pd.read_csv(dataset_path,names=column_names,
                          na_values = "?", comment='\t',
                          sep=" ", skipinitialspace=True)
dataset = raw_dataset.copy()
dataset.tail()
             

In [ ]:
#데이터 정제하기 
dataset.isna().sum() #isna() na 결측값 찾는 함수


In [ ]:
dataset = dataset.dropna()

In [ ]:
origin = dataset.pop('Origin') # 범주형 데이터 원-핫 인코딩으로 변환

In [ ]:
dataset['USA'] = (origin==1)*1.0
dataset['Europe'] = (origin==2)*1.0
dataset['Japan'] = (origin==3)*1.0
dataset.tail()

In [ ]:
#데이터셋을 훈련 세트와 테스트 세트로 분할하기

train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
#데이터 조사하기 - 훈련 세트에서 몇 개의 열을 선택해 산점도 행렬을 만들어 살펴 보기
sns.pairplot(train_dataset[["MPG","Cylinders","Displacement","Weight"]],diag_kind="kde")

In [ ]:
train_dataset[["MPG","Cylinders","Displacement","Weight"]]

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()
train_stats

In [ ]:
#특성과 레이블 분리하기

train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

In [ ]:
#데이터 정규화
def norm(x):
    return (x-train_stats['mean'])/train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
#모델 만들기
def build_model():
    model = keras.Sequential([
        layers.Dense(64,activation='relu', input_shape=[len(train_dataset.keys())]),
        layers.Dense(64,activation='relu'),
        layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mse',
              optimizer=optimizer,
              metrics=['mae','mse'])
    return model

In [ ]:
model = build_model()

In [ ]:
#모델 확인
model.summary()

In [ ]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

In [ ]:
# 모델 훈련
# 에포크가 끝날 때 마다 점(.)을 출력해 훈련 진행 과정을 표시
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs):
        if epoch % 100 == 0: print('')
        print('.',end='')
EPOCHS = 1000

history = model.fit(
    normed_train_data, train_labels,
    epochs=EPOCHS, validation_split = 0.2, verbose=0,
    callbacks=[PrintDot()])

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
import matplotlib.pyplot as plt
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] =history.epoch

    plt.figure(figsize=(8,12))

    plt.subplot(2,1,1)
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [MPG]')
    plt.plot(hist['epoch'],hist['mae'],
             label='Train Error')
    plt.plot(hist['epoch'],hist['val_mae'],
             label='Val Error')
    plt.ylim([0,5])
    plt.legend()

    plt.subplot(2,1,2)
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error [$MPG^2$]')
    plt.plot(hist['epoch'],hist['mse'],
             label='Train Error')
    plt.plot(hist['epoch'],hist['val_mse'],
             label='Val Error')
    
    plt.ylim([0,20])
    plt.legend()
    plt.show()

plot_history(history)
    

In [ ]:
model = build_model()

#patience 매개변수는 성능 향상을 체크할 에포크 횟수

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',patience=10)
history = model.fit(normed_train_data,train_labels,epochs=EPOCHS,
                    validation_split = 0.2 ,verbose=0, callbacks=[early_stop,PrintDot()])
plot_history(history)

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data , test_labels, verbose=2)

print("테스트 세트의 평균 절대 오차: {:5.2f} MPG".format(mae))

In [ ]:
#예측
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels,test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_=plt.plot([-100,100],[-100,100])

In [ ]:
error =test_predictions-test_labels
plt.hist(error,bins=25)
plt.xlabel("Prediction Error [MPG]")
_=plt.ylabel("Count")